In [1]:
import os
import numpy as np

In [2]:
base_path="/Users/zhouwencheng/Desktop/Grass/data/txt/letters/"
source_path= base_path+"letters_source.txt"
target_path= base_path+"letters_target.txt"

In [3]:
with open(source_path) as f:
    source_texts=f.read().split('\n')
with open(target_path) as f:
    target_texts=f.read().split('\n')

In [4]:
pad_flag='<PAD>'
unk_flag='<UNK>'
start_flag='<GO>'
end_flag='<EOS>'
flag_list=[pad_flag, unk_flag, start_flag, end_flag]

all_chars=list(set("".join(source_texts)+"".join(target_texts)))
all_chars=flag_list+sorted(all_chars)
index_to_char={i:char for i, char in enumerate(all_chars)}
char_to_index={char:i for i, char in enumerate(all_chars)}
max_len_source=max([len(item) for item in source_texts])
max_len_traget=max([len(item) for item in target_texts])


pad_flag_index=char_to_index[pad_flag]
unk_flag_index=char_to_index[unk_flag]
start_flag_index=char_to_index[start_flag]
end_flag_index=char_to_index[end_flag]


In [5]:
# pad
id_sources=[]
for item in source_texts:
    pad_len=max_len_source-len(item)
    item_paded=[char_to_index.get(char, unk_flag_index)  for char in item]+[end_flag_index]+[pad_flag_index]*pad_len
    id_sources.append(item_paded)
id_target=[]
for item in target_texts:
    pad_len=max_len_traget-len(item)
    item_paded=[start_flag_index]+[char_to_index.get(char, unk_flag_index)  for char in item]+[end_flag_index]+[pad_flag_index]*pad_len
    id_target.append(item_paded)
    
vocab_len=len(index_to_char)
source_input_len=max([len(item) for item in id_sources])
traget_input_len=max([len(item) for item in id_target])

In [6]:
# target_enbeding
z_target= np.zeros((len(id_target), traget_input_len, vocab_len))
for i in range(len(id_target)):
    item=id_target[i]  
    for j in range(len(item)):
        index=item[j] 
        z_target[i, j, index]=1

In [9]:
from seq2seq import AttentionSeq2Seq
from keras.layers import Embedding, Dense, Activation, Flatten
from keras.layers import BatchNormalization
from keras.models import Sequential

In [10]:
enc_vocab_size = vocab_len
dec_vocab_size = vocab_len
enc_input_length = source_input_len
dec_output_length = traget_input_len
enc_embedding_length = 36
dec_embedding_length = 36

In [12]:
embedding_layer = Embedding(
        vocab_len,
        enc_embedding_length, 
        input_length=enc_input_length,
        trainable=True,
        name='encoder_embedding')
enc_normalization = BatchNormalization()
seq2seq = AttentionSeq2Seq(
        bidirectional=False,
        output_dim=dec_embedding_length, 
        hidden_dim=128, 
        output_length=dec_output_length, 
        input_shape=(enc_input_length, enc_embedding_length), 
        depth=2)
model = Sequential()
model.add(embedding_layer)
model.add(enc_normalization)
model.add(seq2seq)
# model.add(Flatten())
model.add(Dense(dec_vocab_size))
model.add(Activation('softmax'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.summary()

encoded.shape: (?, ?, 128)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_embedding (Embedding (None, 8, 36)             1080      
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 36)             144       
_________________________________________________________________
model_36 (Model)             (None, 9, 36)             442185    
_________________________________________________________________
dense_86 (Dense)             (None, 9, 30)             1110      
_________________________________________________________________
activation_176 (Activation)  (None, 9, 30)             0         
Total params: 444,519
Trainable params: 444,447
Non-trainable params: 72
_________________________________________________________________


In [27]:
from keras.models import load_model
 
model.load_weights('letter_model.h5')
# model = load_model('letter_model.h5')

id_sources=np.array(id_sources)
id_target=np.array(id_target)

model.fit(x=id_sources, y=z_target, batch_size=32, epochs=40)
# model.save('letter_model.h5')
model.save_weights("letter_model.h5", overwrite=True)

Epoch 1/40
10000/10000 [==============================] - 26s 3ms/step - loss: 0.0058 - acc: 0.8549
Epoch 2/40
10000/10000 [==============================] - 25s 3ms/step - loss: 0.0052 - acc: 0.8704
Epoch 3/40
10000/10000 [==============================] - 22s 2ms/step - loss: 0.0052 - acc: 0.8721
Epoch 4/40
10000/10000 [==============================] - 25s 3ms/step - loss: 0.0049 - acc: 0.8824
Epoch 5/40
10000/10000 [==============================] - 26s 3ms/step - loss: 0.0046 - acc: 0.8898
Epoch 6/40
10000/10000 [==============================] - 26s 3ms/step - loss: 0.0043 - acc: 0.8984
Epoch 7/40
10000/10000 [==============================] - 23s 2ms/step - loss: 0.0038 - acc: 0.9130
Epoch 8/40
10000/10000 [==============================] - 22s 2ms/step - loss: 0.0037 - acc: 0.9147
Epoch 9/40
10000/10000 [==============================] - 22s 2ms/step - loss: 0.0035 - acc: 0.9214
Epoch 10/40
10000/10000 [==============================] - 22s 2ms/step - loss: 0.0031 - acc: 0.9316

In [43]:
test="a"
pad_len=max_len_source-len(test)
item_paded=[char_to_index.get(char, unk_flag_index)  for char in test]+[end_flag_index]+[pad_flag_index]*pad_len
test_id=[item_paded]
test_id=np.array(test_id)
test_id.shape

(1, 8)

In [44]:
prediction = model.predict_on_batch(test_id)[0]
prediction.shape
for item in prediction:
    idx = np.argmax(item, axis=0)
    char = index_to_char.get(idx, unk_flag)
    print(char)

<GO>
a
<EOS>
<PAD>
<PAD>
<PAD>
<PAD>
<PAD>
<PAD>


In [45]:
model